In [1]:
import pandas as pd
import regex as re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/ranggi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data MUA for Memory Based System Recommendation

In [2]:
class ContentBasedFiltering:
    def case_folding(self, text):
        if type(text) == str:
            text = text.lower()
        return text

    def punctuation_removal(self, text):
        if type(text) == str:
            text = re.sub(r'[^\w\s]', '', text)
        return text

    def tokenize(self, text):
        tokens = word_tokenize(text)
        return tokens
        
    def stopwords_removal(self, text):
        stopword_factory = StopWordRemoverFactory()
        stopword_remover = stopword_factory.create_stop_word_remover()
        no_stopword = stopword_remover.remove(text)
        return no_stopword

    def preprocessing(self, text):
        text = self.case_folding(text)
        text = self.punctuation_removal(text)
        text = self.stopwords_removal(text)
        return text

In [3]:
data = pd.read_excel('./web_mua/static/Data MUA 2.xlsx')
data.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating
0,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,natural (putih),light,cool,Murah,-8.233613,113.66733,4
1,1,Anggia Pradita A,Intan Rias Pengantin,-8.233613,113.686384,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,4
2,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,rachel 1 (kuning langsat),medium,neutral,Murah,-8.233613,113.66733,5
3,2,VIKA APRILIA SINTA PUTRI,Sanggar Rias Cantik Tina,-8.221109,113.680964,la tulipe cover foundation,suntan (sawo matang),medium,warm,Murah,-8.233613,113.66733,5
4,3,septia maharani,Rias Pengantin Azizah,-8.234863,113.680670,la tulipe cover foundation,natural (putih),light,cool,Sedang,-8.233613,113.66733,3


In [4]:
def cleaning_data(data):
    cleaned_data = []
    CBF = ContentBasedFiltering()
    for i in range(len(data)):
        cleaned_data.append(CBF.preprocessing(str(data[i])))
    return cleaned_data

In [5]:
#praproses data untuk nama
data['nama'] = cleaning_data(data['nama'])

In [6]:
data_for_model = data.copy()
data_for_user = data.copy()

#praproses data untuk deskripsi
data_for_model['desc'] = data['produk_makeup'] + ' ' + data['shade'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_model['desc'] = cleaning_data(data_for_model['desc'])
data_for_user['desc'] = data['produk_makeup'] + ' ' + data['skin_color'] + ' ' + data['skin_undertone']
data_for_user['desc'] = cleaning_data(data_for_user['desc'])

In [7]:
# data_for_model = data_for_model.groupby('no').agg({'nama':'first', 'nama_MUA':'first', 'kategori_harga': 'first', 'latitude':'first', 'longtitude':'first', 'rating':'first','desc':' '.join}).reset_index()
data_for_model = data_for_model.sort_values(by=['nama'], ignore_index=True)

data_for_user = data_for_user.sort_values(by=['nama'], ignore_index=True)
data_for_user = data_for_user.drop(columns=['shade'])

In [8]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [9]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,Sedang,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,Sedang,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,Sedang,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,Sedang,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


In [10]:
data_for_model.shape

(179, 14)

In [11]:
from sklearn.preprocessing import LabelEncoder

harga_labels = LabelEncoder()
harga_labels = harga_labels.fit(data['kategori_harga'])
harga_labels.classes_ = np.array(['Murah', 'Sedang', 'Mahal'])
data_for_model['kategori_harga'] = harga_labels.transform(data_for_model['kategori_harga'])
data_for_user['kategori_harga'] = harga_labels.transform(data_for_user['kategori_harga'])

In [12]:
data_for_model.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation natur...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation suntan sawo matang ...
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,suntan (sawo matang),medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation suntan sawo matang ...
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,plum rose (putih),light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation plum ...


In [13]:
data_for_user.head()

,no,nama,nama_MUA,latitude,longitude,produk_makeup,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
0,53,adera zia,Dinda Wijaya Make Up,-8.193711,113.619405,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.193711,113.616830,5,lt pro smooth corrector cream foundation mediu...
1,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation mediu...
2,79,adera zia,Bonita MUA Jember,-8.139019,113.731123,la tulipe cover foundation,medium,warm,1,-8.139019,113.728548,4,la tulipe cover foundation medium warm
3,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,la tulipe cover foundation,medium,warm,1,-8.157255,113.652074,3,la tulipe cover foundation medium warm
4,48,adera zia,Ardie Make Up & Salon,-8.144589,113.719387,LT Pro smooth corrector cream foundation,light,cool,1,-8.157255,113.652074,3,lt pro smooth corrector cream foundation light...


## User Input

In [14]:
# use this parameter to get some MUA recommendations
def data_preparation(data_for_model, data_for_user, index_user, user):
    du = data_for_user.copy()
    query_data = du.iloc[[index_user]].reset_index()
    query_data_todict = query_data.to_dict('index')[0]
    part_of_model = data_for_model.copy()
    part_of_model = part_of_model.loc[part_of_model.nama_MUA != query_data.nama_MUA[0]].reset_index()
    part_of_query = part_of_model.loc[part_of_model['nama'] == user].reset_index()
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return query_data_todict, part_of_query, part_of_model

# example of input data
find_user = data_for_user.index[data_for_user['nama'] == 'adera zia'].to_list()
txt_user = 'adera zia'
query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user)
# part_of_model = data_preparation(data_for_model, data_for_user, find_user[0], txt_user, first=False)

## Calculating Cosine Similarity for Content Based Model

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

def tf_idf_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(md["desc"])
    model = X[:len(part_of_model)]
    query = X[len(part_of_model):]
    return model, query

def doc2vec_vector(part_of_model, query_data_todict):
    copy_md = part_of_model.copy()
    md = pd.DataFrame(copy_md["desc"])
    md.loc[len(md)] = query_data_todict["desc"]
    words_split = []
    for i in range(len(md)):
        words_split.append([word for word in md["desc"][i].split()])
    documents = []
    for i, list_of_words in enumerate(words_split):
        documents.append(gensim.models.doc2vec.TaggedDocument(words=list_of_words, tags=['doc'+str(i)]))
    model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=30)
    # vectorizer = TfidfVectorizer()
    # X = vectorizer.fit_transform(md["desc"])
    # nmf_model = NMF(10)
    # doc_topic = nmf_model.fit_transform(X)
    vec_model = []
    for i in range((len(part_of_model))):
        vec_model.append(model.infer_vector(md["desc"][i].split()))
    vec_query = model.infer_vector(md["desc"][len(md["desc"])-1].split())
    # print(vec_query)
    return vec_model, vec_query

In [16]:
from geopy.distance import great_circle
import math
def calculate_distance(query_data_todict, part_of_model):
        loc1 = (query_data_todict['latitude'], query_data_todict['longtitude'])
        distance_loc = []
        for i in range(len(part_of_model.index)):
            loc2 = (part_of_model['latitude'][i], part_of_model['longtitude'][i])
            tmp = great_circle(loc1, loc2).km
            value = math.ceil(tmp * 100) / 100
            distance_loc.append(value)
        return distance_loc

In [17]:
from gensim import similarities
from scipy import spatial 
import torch

def doc2vec_model(sentences1, sentences2, model):
    res = []
    for sentence1, sentence2 in zip(sentences1, sentences2):
        res.append(model.similarity_unseen_docs(sentence1, sentence2))
    return torch.tensor(res)

def get_numtext_content(query_data_todict, part_of_model, tfidf=False):
    if tfidf == False:
        vec_model, vec_query = doc2vec_vector(part_of_model, query_data_todict)
        cos_sim = []
        for i in range(len(vec_model)):
            cos_sim.append(spatial.distance.cosine(vec_model[i], vec_query))
        distance = calculate_distance(query_data_todict, part_of_model)
        return cos_sim, distance
    else:
        tfidf_model, tfidf_query = tf_idf_vector(part_of_model, query_data_todict)
        cos_sim = cosine_similarity(tfidf_query, tfidf_model)
        data_sim = cos_sim.reshape(-1,1)
        distance = calculate_distance(query_data_todict, part_of_model)
        return data_sim, distance

In [18]:
# x, y = get_numtext_content(query_data_todict, part_of_model)

In [19]:
from sklearn.preprocessing import StandardScaler

def get_values(part_of_model):
    tmp_data = part_of_model[['kategori_harga','latitude', 'longitude']].to_numpy()
    sim_num = cosine_similarity(tmp_data)
    sim_num = np.array([np.mean(p) for p in sim_num])
    return sim_num

In [20]:
def value_of_mua(query_data_todict, part_of_model, tfidf=False):
    sim, distance = get_numtext_content(query_data_todict, part_of_model, tfidf)
    part_of_model["sim"] = sim
    part_of_model["distance"] = distance
    sim_num = get_values(part_of_model)
    part_of_model["sim_num"] = sim_num
    part_of_model = part_of_model.sort_values(by=['nama_MUA'])
    nama_mua = part_of_model["nama_MUA"].unique()
    value_mua = np.empty(len(nama_mua), dtype=float)
    for i in range(len(nama_mua)):
        tmp_val = []
        for j in range(len(part_of_model.index)):
            if nama_mua[i] == part_of_model.nama_MUA[j]:
                tmp_val.append(part_of_model["sim_num"][j] + part_of_model["sim"][j])
        value_mua[i] = np.mean(tmp_val)
    part_of_model = part_of_model.reset_index()
    part_of_model = part_of_model.drop(columns='index')
    return nama_mua, value_mua, part_of_model

## FIND USER

In [21]:
unique_user = data_for_model['nama'].unique()
unique_user
query_user = ['adera zia', 'anggia pradita a', 'anissa serina', 'calista devi aqilah', 'callista febriana erwanto', 'dwi rahma', 'maulina vira', 
              'vika aprilia sinta putri', 'yuricha mahargiyani wijaya', 'zainata hanum masruroh', 'zyanti laila']
selected_user = []
for i in query_user:
    find_user = data_for_model.index[data_for_model['nama'] == i].to_list()
    selected_user.append(find_user[0])
selected_user

[0, 10, 20, 33, 45, 73, 101, 142, 151, 161, 169]

## HYBRID FILTERING

In [22]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing as pre
import statistics 

In [89]:
#chosen user for testing
from sklearn.cluster import KMeans
def hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user, tfidf=False):
    actual_hyb = []
    table_hyb = []
    rating_hyb = []
    for index_user in range(len(selected_user)):
        query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model, tfidf)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        
        cos_sim_user = cosine_similarity([feature], val_model)

        weight_user = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_user)):
                if part_of_model["nama"][i] == nama_user[j]:
                    weight_user[i] = cos_sim_user[0][j]
        
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA']]
        final_res['user'] = weight_user
        final_res['content'] = weight_content

        fvector = []
        for i in range(len(final_res['nama'])):
            fvector.append([final_res['user'][i], final_res['content'][i]])
        X = np.array(fvector)
        kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto").fit(X)
        final_res['cluster'] = kmeans.labels_
        getLabel = kmeans.labels_[selected_user[index_user]]
        print(getLabel)
        # mean_weight = np.zeros(len(part_of_model), dtype=float)
        # for i in range(len(part_of_model)):
        #     mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
        # final_res['mean_'] = mean_weight
        getUser = []
        for i in range(len(final_res['nama'])):
            if final_res['cluster'][i] == getLabel:
                if final_res['nama'][i] not in getUser:
                    getUser.append(final_res['nama'][i])

        # higher_value = final_res.copy()
        # higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        # u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
        # higher_value['weight_user'] = u['mean']
        # c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
        # higher_value['weight_content'] = c['mean']
        # higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
        # list_nama = higher_value.nama[:15].tolist()
        pivot = part_of_model.copy()
        pivot['content'] = weight_content
        print(getUser)
        # pivot['mean_'] = mean_weight
        pivot = pivot[pivot.nama.isin(getUser)].reset_index()
        print(pivot)
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_2 = pivot.groupby("nama_MUA").content.agg(['count','mean'])
        # pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        print(weight_content)
        pivot_['predicted'] = pivot_2['mean']
        pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = np.mean([pivot_["mean"][i] + pivot_2['mean'][i]])
                else:
                    continue
        
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break

        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        
        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_hyb.append(final_label)
        table_hyb.append(pivot_)
        rating_hyb.append(value_rating)

    #     mae = mean_absolute_error(value_rating, final_actual[index_user])
    #     mae_hy.append(mae)
        
    # mean_mae_hy = np.mean(mae_hy)
    return table_hyb, rating_hyb, actual_hyb, col_model

In [32]:
hybrid_based_fil_doc2vec, value_rating_hyb_doc2vec, actual_hyb_doc2vec, col_model_doc2vec = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user)

ValueError: operands could not be broadcast together with shapes (3,) (173,) 

In [90]:
hybrid_based_fil_tfidf, value_rating_hyb_tfidf, actual_hyb_tfidf, col_model_tfidf = hybrid_evaluation(data_for_model, data_for_user, selected_user, query_user, tfidf=True)

0
['anggun yeriam', 'dillah ayu firnanda', 'denada ilma', 'maryam kafila', 'yuricha mahargiyani wijaya', 'priska sisilia dewi', 'frensisca amelia', 'calista devi aqilah', 'dwinta nasya amelia', 'eka febrianti', 'olivia ramadhania', 'moh wahyu nur hidayah', 'diajeng agustin', 'septia maharani', 'zahra salsabila bilqis', 'rahma kamilla nur imam', 'azwa sasa', 'rodhiyyatul hikmah', 'nadia safitri', 'yuna ayana', 'azahra aura wiritanaya', 'sonia arista firnanda', 'vika aprilia sinta putri']
    index  level_0  no                        nama  \
0       0       17  65               anggun yeriam   
1       1       64  70         dillah ayu firnanda   
2       2       65  70         dillah ayu firnanda   
3       3       16  65               anggun yeriam   
4       6       58  68                 denada ilma   
5       7       59  68                 denada ilma   
6       8       98  67               maryam kafila   
7      21      149  23  yuricha mahargiyani wijaya   
8      22      150  23

In [91]:
value_rating_hyb_tfidf[0]

array([6.25408617, 0.        , 5.64152992, 0.        , 0.        ,
       0.        , 6.30513849, 0.        , 5.53392816, 0.        ,
       6.07009822, 0.        , 6.32291799, 5.22262789, 0.        ,
       0.        , 0.        , 0.        , 6.50503979, 0.        ,
       5.31684886, 0.        , 5.36782707, 0.        , 5.07187327,
       0.        , 5.37731298, 5.33751267, 5.38186163, 4.27250636,
       0.        , 6.06280145, 5.32944626, 5.74762422, 0.        ,
       0.        , 3.27256909, 4.2719087 , 0.        , 0.        ,
       6.17332848, 6.37731259, 4.50503992, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       6.24512982])

## COLLABORATIVE FILTERING

In [65]:
def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
    actual_colab = []
    table_colab = []
    rating_colab = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        val_model = np.array(pivot.values.tolist(), dtype=float)
        nama_user = np.array(pivot.index.tolist())
        pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_query = pivot_query.values.tolist()
        col_query = pivot_query.columns.tolist()
        feature = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_query)):
                if col_model[i] == col_query[j]:
                    feature[i] = val_query[0][j]
                    break
        
        cos_sim_user = cosine_similarity([feature], val_model)
        
        final_res = part_of_model.copy()
        final_res = final_res[['nama', 'nama_MUA', 'rating']]
        tmp_cos = np.zeros(len(part_of_model), dtype=float)
        for i in range(len(final_res)):
            for j in range(len(nama_user)):
                if final_res['nama'][i] == nama_user[j]:
                    tmp_cos[i] = cos_sim_user[0][j]
                    break

        final_res['mean_'] = tmp_cos
        higher_value = final_res.copy()
        higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
        list_nama = higher_value.nama[:15].tolist()
        pivot = final_res.copy()
        pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
        pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_['weight']
        pivot_ = pivot_.sort_values(by=["predicted"], ascending=False)
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["weight"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_col.append(mae)

        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_colab.append(final_label)
        table_colab.append(pivot_)
        rating_colab.append(value_rating)
        
    # mean_mae_col = np.mean(mae_col)
    return table_colab, rating_colab, actual_colab

In [83]:
# def colab_evaluation(data_for_model, data_for_user, selected_user, query_user):
#     actual_colab = []
#     table_colab = []
#     rating_colab = []
#     for index_user in range(len(selected_user)):
#         _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
#         nama_mua, _, part_of_model = value_of_mua(query_data_todict, part_of_model)
#         pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         col_model = pivot.columns.tolist()
#         val_model = np.array(pivot.values.tolist(), dtype=float)
#         nama_user = np.array(pivot.index.tolist())
#         pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         val_query = pivot_query.values.tolist()
#         col_query = pivot_query.columns.tolist()
#         feature = np.zeros(len(col_model), dtype=float)
#         for i in range(len(col_model)):
#             for j in range(len(col_query)):
#                 if col_model[i] == col_query[j]:
#                     feature[i] = val_query[0][j]
#                     break
#         knn = NearestNeighbors(metric='cosine', algorithm='brute')
#         knn.fit(val_model)
#         distances, indices = knn.kneighbors([feature], n_neighbors=10)
#         list_nama = []
#         for ln in indices[0]:
#              list_nama.append(nama_user[ln])
#         pivot = data_for_model[data_for_model.nama.isin(list_nama)].reset_index()
#         tmp_cos = np.empty(len(pivot), dtype=float)
#         for i in range(len(pivot.nama)):
#             for j in range(len(list_nama)):
#                 if pivot.nama[i] == list_nama[j]:
#                     tmp_cos[i] = distances[0][j]
#         pivot['cos'] = tmp_cos
#         pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
#         pivot_['cos'] = pivot.groupby("nama_MUA").cos.agg(['mean'])
#         pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
#         pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
#         value_rating = np.zeros(len(col_model), dtype=float)
#         for i in range(len(pivot_.index)):
#             for j in range(len(col_model)):
#                 if value_rating[j] == 0.0:
#                     if col_model[j] == pivot_.index[i]:
#                         value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
#                 else:
#                     continue
#         tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
#         mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
#         pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
#         val_label = pivot_label.values.tolist()
#         col_label = pivot_label.columns.tolist()
#         final_label = np.zeros(len(col_model), dtype=float)
#         for i in range(len(col_model)):
#             for j in range(len(col_label)):
#                 if col_model[i] == col_label[j]:
#                     final_label[i] = val_label[0][j]
#                     break
#         # mae = mean_absolute_error(value_rating, final_actual[index_user])
#         # mae_col.append(mae)

#         for i in range(len(final_label)):
#             if final_label[i] == 0.0:
#                 for j in range(len(pivot_['mean'])):
#                     if col_model[i] == pivot_.index.tolist()[j]:
#                         final_label[i] = pivot_['mean'].tolist()[j]
#         actual_colab.append(final_label)
#         table_colab.append(pivot_)
#         rating_colab.append(value_rating)
        
#     # mean_mae_col = np.mean(mae_col)
#     return table_colab, rating_colab, actual_colab

In [66]:
colab_based_fil, value_rating_colab, actual_colab = colab_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [67]:
# colab_based_fil[colab_based_fil.score > 0.0]

In [68]:
colab_based_fil[0]

,count,mean,weight,predicted
nama_MUA,,,,
Bonita MUA Jember,2,4.000000,1.000000,4.000000
Larasnada Make Up,3,3.666667,0.735653,2.697393
Ardie Make Up & Salon,4,3.500000,0.593704,2.077965
Laura Make Up & Bridal,1,5.000000,0.206958,1.034790
Eka Make Over,3,5.000000,0.187409,0.937043
Rias Manten Mariyam,3,5.000000,0.187409,0.937043
Liyan Make Up Jember,1,4.000000,0.206958,0.827832
Ulfa Beauty MUA,1,4.000000,0.206958,0.827832
Rias Pengantin Criepno Lestari,3,3.000000,0.187409,0.562226


## CONTENT BASED FILTERING

In [69]:
def content_evaluation(data_for_model, data_for_user, selected_user, query_use):
    actual_content = []
    table_content = []
    rating_content = []
    for index_user in range(len(selected_user)):
        _, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_user], query_user[index_user])
        nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
        pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        col_model = pivot.columns.tolist()
        weight_content = np.empty(len(part_of_model), dtype=float)
        for i in range(len(part_of_model)):
            for j in range(len(nama_mua)):
                if part_of_model["nama_MUA"][i] == nama_mua[j]:
                    weight_content[i] = value_mua[j]

        pm = part_of_model.copy()
        pm["cos"] = weight_content
        pivot_ = pm.groupby("nama_MUA").rating.agg(['count','mean'])
        pivot_['cos'] = pm.groupby("nama_MUA").cos.agg(['mean'])
        pivot_['predicted'] = pivot_["mean"] * pivot_["cos"]
        pivot_ = pivot_.sort_values(by=['predicted'], ascending=False)
        pivot_ = pivot_[:15]
        value_rating = np.zeros(len(col_model), dtype=float)
        for i in range(len(pivot_.index)):
            for j in range(len(col_model)):
                if value_rating[j] == 0.0:
                    if col_model[j] == pivot_.index[i]:
                        value_rating[j] = pivot_["mean"][i] * pivot_["cos"][i]
                else:
                    continue
        tmp_data_recs = {'nama_mua':nama_mua, 'score':value_rating}
        mua_recs = pd.DataFrame(tmp_data_recs).sort_values(by=['score'], ignore_index=True, ascending=False)
        pivot_label = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
        val_label = pivot_label.values.tolist()
        col_label = pivot_label.columns.tolist()
        final_label = np.zeros(len(col_model), dtype=float)
        for i in range(len(col_model)):
            for j in range(len(col_label)):
                if col_model[i] == col_label[j]:
                    final_label[i] = val_label[0][j]
                    break
        # mae = mean_absolute_error(value_rating, final_actual[index_user])
        # mae_con.append(mae)

        for i in range(len(final_label)):
            if final_label[i] == 0.0:
                for j in range(len(pivot_['mean'])):
                    if col_model[i] == pivot_.index.tolist()[j]:
                        final_label[i] = pivot_['mean'].tolist()[j]
        actual_content.append(final_label)
        table_content.append(pivot_)
        rating_content.append(value_rating)
        
    # mean_mae_con = np.mean(mae_con)
    return table_content, rating_content, actual_content

In [70]:
content_based_fil, value_rating_content, actual_content = content_evaluation(data_for_model, data_for_user, selected_user, query_user)

In [71]:
# content_based_fil[content_based_fil.score > 0.0]

In [72]:
content_based_fil[0]

,count,mean,cos,predicted
nama_MUA,,,,
Gasebo Make Up Dekorasi,4,5.000000,1.060535,5.302676
Aira Salon,2,5.000000,1.056113,5.280564
Rias Pengantin Sapphire,4,5.000000,1.043919,5.219593
Khaliza Make Up,3,5.000000,1.041057,5.205287
Wedding Gallery Make Over,2,5.000000,1.033809,5.169044
Aulia Rias Pengantin,4,5.000000,1.032267,5.161337
Sanggar Rias Nining,5,5.000000,1.031708,5.158539
RR Make Up,3,5.000000,1.030565,5.152823
Winda Wijaya MUA & Beauty Studio,3,5.000000,1.029963,5.149814


In [75]:
merge_actual = [[i, j, z] for i,j,z in zip(actual_colab, actual_content, actual_hyb_tfidf)]
final_actual = []
for i in range(len(merge_actual)):
    tmp = []
    for j in range(len(merge_actual[i][0])):
        sfmean = statistics.fmean([merge_actual[i][0][j], merge_actual[i][1][j], merge_actual[i][2][j]])
        tmp.append(round(sfmean))
    final_actual.append(tmp)
    

In [76]:
def mae_evaluation(predicted_data, actual_data, hybrid=False):
    maeUser = []
    rmseUser = []

    for i in range(len(predicted_data)):
        mae = mean_absolute_error(predicted_data[i], actual_data[i])
        rmse = mean_squared_error(predicted_data[i], actual_data[i], squared=False)
        maeUser.append(mae)
        rmseUser.append(rmse)
    mae = np.mean(maeUser)
    rmse = np.mean(rmseUser)
    if hybrid == True:
        return mae, rmse, maeUser
    else:
        return mae, rmse

In [77]:
mae_col, rmse_col = mae_evaluation(value_rating_colab, final_actual)
mae_con, rmse_con = mae_evaluation(value_rating_content, final_actual)
# mae_hyb, rmse_hyb, mae = mae_evaluation(value_rating_hyb_doc2vec, final_actual, hybrid=True)
mae_hyb_tfidf, rmse_hyb_tfidf, mae_tfidf = mae_evaluation(value_rating_hyb_tfidf, final_actual, hybrid=True)

In [79]:
mae_tfidf

[1.9299326015533262,
 2.28908602452493,
 2.47691617702771,
 2.358864553978589,
 1.9846171884068797,
 1.6914387985177222,
 2.213437659102279,
 1.9779600009899747,
 1.123879097454525,
 2.769706424634222,
 1.5991415351367848]

In [80]:
data_evaluasi = {
    'Metode' : ['Collaborative Filtering', 'Content Based Filtering', 'Hybrid Filtering'],
    'MAE' : [mae_col, mae_con, mae_hyb_tfidf],
    'RMSE' : [rmse_col, rmse_con, rmse_hyb_tfidf]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE,RMSE
0,Content Based Filtering,1.523371,1.970309
1,Collaborative Filtering,1.749986,2.245621
2,Hybrid Filtering,2.037725,2.501039


In [95]:
mae_hyb_tfidf, rmse_hyb_tfidf, mae_tfidf = mae_evaluation(value_rating_hyb_tfidf, final_actual, hybrid=True)
data_evaluasi_hyb = {
    'Metode' : ['TFIDF', 'Doc2vec'],
    'MAE' : [mae_hyb_tfidf, mae_hyb],
    'RMSE' : [rmse_hyb_tfidf, rmse_hyb]
}

tabel_evaluasi = pd.DataFrame(data_evaluasi_hyb)
tabel_evaluasi = tabel_evaluasi.sort_values(by=['MAE'], ignore_index=True)
tabel_evaluasi

,Metode,MAE,RMSE
0,Doc2vec,0.986916,1.363108
1,TFIDF,1.191766,1.630573


In [96]:
def testing_user(data_for_model, data_for_user, index_usr):
    query_data_todict, part_of_query, part_of_model = data_preparation(data_for_model, data_for_user, selected_user[index_usr], query_user[index_usr])
    nama_mua, value_mua, part_of_model = value_of_mua(query_data_todict, part_of_model)
    pivot = pd.pivot_table(part_of_model, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    col_model = pivot.columns.tolist()
    val_model = np.array(pivot.values.tolist(), dtype=float)
    nama_user = np.array(pivot.index.tolist())
    pivot_query = pd.pivot_table(part_of_query, columns='nama_MUA', index='nama', values='rating', aggfunc='mean', fill_value=0)
    val_query = pivot_query.values.tolist()
    col_query = pivot_query.columns.tolist()
    feature = np.zeros(len(col_model), dtype=float)
    for i in range(len(col_model)):
        for j in range(len(col_query)):
            if col_model[i] == col_query[j]:
                feature[i] = val_query[0][j]
                break
    
    # nmf_model = NMF(3)
    # f = nmf_model.fit_transform([feature])
    # m = nmf_model.fit_transform(val_model)
    cos_sim_user = cosine_similarity([feature], val_model)

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]

    weight_user = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_user)):
            if part_of_model["nama"][i] == nama_user[j]:
                weight_user[i] = cos_sim_user[0][j]
        
    weight_content = np.empty(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        for j in range(len(nama_mua)):
            if part_of_model["nama_MUA"][i] == nama_mua[j]:
                weight_content[i] = value_mua[j]
        
    final_res = part_of_model.copy()
    final_res = final_res[['nama', 'nama_MUA']]
    final_res['user'] = weight_user
    final_res['content'] = weight_content

    mean_weight = np.zeros(len(part_of_model), dtype=float)
    for i in range(len(part_of_model)):
        mean_weight[i] = statistics.harmonic_mean([weight_user[i] + np.min(weight_user[np.nonzero(weight_user)]),  weight_content[i]]) 
    final_res['mean_'] = mean_weight
    higher_value = final_res.copy()
    higher_value = final_res.groupby("nama", as_index=False).mean_.agg(["mean"])
    u = final_res.groupby("nama", as_index=False).user.agg(["mean"])
    higher_value['weight_user'] = u['mean']
    c = final_res.groupby("nama", as_index=False).content.agg(["mean"])
    higher_value['weight_content'] = c['mean']
    higher_value = higher_value.sort_values(by=['mean'], ignore_index=True, ascending=False)
    print(higher_value)
    list_nama = higher_value.nama[:15].tolist()
    pivot = part_of_model.copy()
    pivot['mean_'] = mean_weight
    pivot = pivot[pivot.nama.isin(list_nama)].reset_index()
    distance = pivot[['nama_MUA','distance']]
    produk = pivot[['nama_MUA', 'sim']]
    desc = pivot[['nama_MUA', 'produk_makeup', 'shade', 'skin_color', 'skin_undertone']]
    desc['desc'] = desc['produk_makeup'] + ' - ' + desc['shade'] + ' - ' + desc['skin_color'] + ' - ' + desc['skin_undertone']
    pivot_ = pivot.groupby("nama_MUA").rating.agg(['count','mean'])
    pivot_['nama_MUA'] = desc.groupby('nama_MUA').first().index.to_list()
    pivot_['weight'] = pivot.groupby("nama_MUA").mean_.agg(['mean'])
    pivot_['score'] = pivot_["mean"] * pivot_['weight']
    pivot_['desc'] = desc.groupby('nama_MUA').agg({'desc': lambda x: list(x)}).reset_index()['desc'].values
    pivot_['distance'] = distance.groupby("nama_MUA").distance.agg(['mean'])
    pivot_['sim'] = produk.groupby("nama_MUA").sim.agg(['mean'])
    pivot_ = pivot_.sort_values(by=['score'], ignore_index=True, ascending=False)
    return pivot_

In [97]:
#highest MAE
mae[mae.index(min(mae))]

0.8408625449576367

In [98]:
#highest
idx_high = mae.index(min(mae))
data_for_model.iloc[[selected_user[idx_high]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
10,17,anggia pradita a,Aulia Rias Pengantin,-8.367258,113.601457,la tulipe cover foundation,natural (putih),light,cool,0,-8.367258,113.601457,5,la tulipe cover foundation natural putih light...


In [99]:
h_pivot = testing_user(data_for_model, data_for_user, idx_high)
h_pivot

                              nama      mean  weight_user  weight_content
0                 anggia pradita a  1.135275     1.000000        1.036383
1                 hikmatul hasanah  0.923687     0.577350        1.037491
2         vika aprilia sinta putri  0.750160     0.333333        1.034708
3              calista devi aqilah  0.742623     0.318788        1.052367
4                     maulina vira  0.688614     0.259828        1.039410
5                    anissa serina  0.684599     0.255031        1.040736
6           azahra aura wiritanaya  0.413066     0.000000        1.086076
7            sonia arista firnanda  0.413066     0.000000        1.086076
8               rodhiyyatul hikmah  0.412430     0.000000        1.077346
9              priska sisilia dewi  0.411572     0.000000        1.065732
10                frensisca amelia  0.411572     0.000000        1.065732
11        dwi anti wulandari putri  0.410087     0.000000        1.046122
12              trevina wigianiska  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim
0,2,4.000000,Intan Rias Pengantin,1.135034,4.540136,[la tulipe cover foundation - natural (putih) ...,16.54,0.036017
1,2,5.000000,Sanggar Rias Cantik Tina,0.751256,3.756279,[la tulipe cover foundation - suntan (sawo mat...,17.80,0.038909
2,6,4.333333,Griya Putri Make Up,0.856973,3.713549,[la tulipe cover foundation - suntan (sawo mat...,24.76,0.035711
3,3,5.000000,Khaliza Make Up,0.737058,3.685290,[la tulipe cover foundation - natural (putih) ...,2.02,0.030118
4,8,4.000000,Yulia Salon & Bridal,0.907470,3.629882,[LT Pro Smooth Corrector Cream Foundation - wa...,0.88,0.037526
5,1,5.000000,Winda Wijaya MUA & Beauty Studio,0.687842,3.439211,[la tulipe cover foundation - suntan (sawo mat...,28.02,0.040570
6,1,5.000000,Laura Make Up & Bridal,0.685116,3.425579,[la tulipe cover foundation - natural (putih) ...,21.89,0.032670
7,3,4.666667,Ulfa Beauty MUA,0.688844,3.214603,[la tulipe cover foundation - suntan (sawo mat...,39.18,0.047065
8,2,4.000000,Hiekmaa MUA,0.747946,2.991785,[la tulipe cover foundation - rachel 2 (kuning...,20.69,0.026349
9,3,4.000000,Novi Make Up,0.745688,2.982750,[LT Pro smooth corrector cream foundation - na...,6.29,0.064553


In [627]:
h_pivot.to_excel('highest2.xlsx')

In [100]:
#lowest MAE
mae[mae.index(max(mae))]

1.1893415973232309

In [101]:
#lowest
idx_low = mae.index(max(mae))
data_for_model.iloc[[selected_user[idx_low]]]

,no,nama,nama_MUA,latitude,longitude,produk_makeup,shade,skin_color,skin_undertone,kategori_harga,latitude.1,longtitude,rating,desc
73,80,dwi rahma,Bonita MUA Jember,-8.139019,113.731123,LT Pro smooth corrector cream foundation,natural (kuning langsat),medium,neutral,1,-8.139019,113.728548,4,lt pro smooth corrector cream foundation natur...


In [102]:
h_pivot = testing_user(data_for_model, data_for_user, idx_low)
h_pivot

                              nama      mean  weight_user  weight_content
0                        dwi rahma  1.104615     1.000000        1.026266
1                     zyanti laila  0.681502     0.312348        1.033743
2         vika aprilia sinta putri  0.567702     0.195959        1.029467
3              siti aisyah humaira  0.330233     0.000000        1.049079
4                  rania salsabila  0.330176     0.000000        1.047911
5                     maulina vira  0.329835     0.000000        1.041106
6                  septia maharani  0.329542     0.000000        1.035275
7                    anissa serina  0.329432     0.000000        1.033141
8            arsya putri rifasanti  0.329411     0.000000        1.032694
9         dwi anti wulandari putri  0.329348     0.000000        1.031455
10          azahra aura wiritanaya  0.329339     0.000000        1.031288
11           sonia arista firnanda  0.329339     0.000000        1.031288
12          zainata hanum masruroh  0.

,count,mean,nama_MUA,weight,score,desc,distance,sim
0,3,4.333333,Ayuk Rias Pengantin,0.827041,3.583843,[LT Pro smooth corrector cream foundation - na...,21.84,0.042879
1,2,3.000000,Iqlim Rias Pengantin,1.101191,3.303572,[LT Pro smooth corrector cream foundation - na...,9.06,0.020358
2,4,3.500000,Sanggar Rias Pengantin Dewi Balqis,0.835061,2.922713,[la tulipe cover foundation - rachel 2 (kuning...,3.90,0.023928
3,2,5.000000,Sanggar Rias Cantik Tina,0.569133,2.845667,[la tulipe cover foundation - rachel 1 (kuning...,11.35,0.038928
4,2,4.000000,Yasmin MUA dan Henna Jember,0.680318,2.721273,[la tulipe cover foundation - rachel 1 (kuning...,11.69,0.028313
5,3,4.333333,Laura Make Up & Bridal,0.564242,2.445047,[la tulipe cover foundation - natural (putih) ...,7.62,0.022961
6,2,4.000000,Hiekmaa MUA,0.566123,2.264490,[la tulipe cover foundation - natural (putih) ...,8.39,0.019141
7,4,4.000000,Dinda Wijaya Make Up,0.504684,2.018735,[Ultima II Wonderwear Cream Make Up - linen - ...,13.72,0.030838
8,5,4.200000,Larasnada Make Up,0.470490,1.976059,[la tulipe cover foundation - rachel 1 (kuning...,2.78,0.035556
9,6,4.333333,Griya Putri Make Up,0.409282,1.773557,[Wardah Everyday Luminous Liquid Foundation - ...,4.52,0.036002


In [631]:
h_pivot.to_excel('lowest2.xlsx')

In [479]:
# rating_test = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = final_actual[i][j]
#     rating_test.append(rating_dict)

# dataTesting = pd.DataFrame.from_dict(rating_test)
# dataTesting['user'] = query_user
# dataTesting.set_index(['user'])
# dataTesting.to_excel('dataTesting2.xlsx')

In [43]:
# predicted_rating = []
# for i in range(len(query_user)):
#     rating_dict = dict()
#     for j in range(len(col_model)):
#         rating_dict[col_model[j]] = value_rating_hyb[i][j]
#     predicted_rating.append(rating_dict)

# predictedRating = pd.DataFrame.from_dict(predicted_rating)
# predictedRating['user'] = query_user
# predictedRating.set_index(['user'])
# predictedRating.to_excel('predictedRating2.xlsx')